This notebook uses curated mappings from Biomappings to assess the degree to which pathway and biological process nodes coming from the Gene Ontology, Reactome, Wikipathways, and NCI-PID nodes are duplicated in Hetionet.

It was demonstrated by Mubeen *et al.* (2019) in [The Impact of Pathway Database Choice on Statistical Enrichment Analysis and Predictive Modeling](https://www.frontiersin.org/articles/10.3389/fgene.2019.01203/full) that merging nodes from different pathway databases representing the same pathways (but with small potential difference in the gene sets annotated for each) is beneficial for downstream analysis.

In [1]:
import itertools as itt
import sys
import time
from collections import Counter

import networkx as nx
import pandas as pd
import pystow
from pyobo.gilda_utils import get_grounder
from tqdm.auto import tqdm

from biomappings.graph import get_true_graph

In [2]:
print(sys.version)

3.11.0 (main, Oct 25 2022, 14:13:24) [Clang 14.0.0 (clang-1400.0.29.202)]


In [3]:
print(time.asctime())

Tue Feb  7 19:04:39 2023


# Data Loading

Load the Biomappings data

In [4]:
true_graph = get_true_graph()

Load the Hetionet data into a data frame

In [5]:
URL = "https://github.com/hetio/hetionet/raw/master/hetnet/json/hetionet-v1.0.json.bz2"
data = pystow.ensure_json_bz2("bio", "hetionet", "1.0", url=URL)

In [6]:
rows = [
    (
        record["kind"],
        record["identifier"],
        record["name"],
        record["data"]["source"],
        # record["data"].get("url"),
        # record["data"].get("description"),
    )
    for record in tqdm(data["nodes"], unit_scale=True, unit="node")
]
df = pd.DataFrame(
    rows,
    columns=["kind", "identifier", "name", "source"],
)
df.head()

  0%|          | 0.00/47.0k [00:00<?, ?node/s]

,kind,identifier,name,source
0,Molecular Function,GO:0031753,endothelial differentiation G-protein coupled ...,Gene Ontology
1,Side Effect,C0023448,Lymphocytic leukaemia,UMLS via SIDER 4.1
2,Gene,5345,SERPINF2,Entrez Gene
3,Gene,9409,PEX16,Entrez Gene
4,Biological Process,GO:0032474,otolith morphogenesis,Gene Ontology


What are the unique kinds of nodes in hetionet?

In [7]:
df.kind.unique()

array(['Molecular Function', 'Side Effect', 'Gene', 'Biological Process',
       'Compound', 'Pathway', 'Anatomy', 'Cellular Component', 'Symptom',
       'Disease', 'Pharmacologic Class'], dtype=object)

# Identify Redundant Pathways

Where do pathways come from?

In [8]:
df[df.kind == "Pathway"].groupby("source").count()["identifier"]

source
PID via Pathway Commons          220
Reactome via Pathway Commons    1308
WikiPathways                     294
Name: identifier, dtype: int64

Biological processes and pathways are the same, but these are split into their own "kind".

In [9]:
df[df.kind == "Biological Process"].count()["identifier"]

11381

## Mapping Pathways from WikiPathways and GO

WikiPathway and GO are possible to map to standard CURIEs with string operations

In [10]:
idx = (df.kind == "Pathway") & (df.source == "WikiPathways")
wikipathways_mapping = {wpid: "wikipathways:" + wpid.split("_")[0] for wpid in df[idx].identifier}

In [11]:
go_mapping = {
    go_curie: go_curie.lower() for go_curie in df[df.kind == "Biological Process"].identifier
}

## Grounding Pathways from Reactome and NCI-PID

Unfortunately, Hetionet imports pathways via Pathway Commons, so its provenance information is mostly lost (i.e., the identifiers are opaque).

In [13]:
def get_mappings(prefix, source):
    grounder = get_grounder(prefix)
    counter = Counter(term.organism for terms in grounder.entries.values() for term in terms)
    print(f"{prefix.capitalize()} species count: {counter}")

    idx = (df.kind == "Pathway") & (df.source == source)
    it = tqdm(df[idx].values, unit_scale=True, desc="Grounding")
    rv = {}
    for _, identifier, name, _ in it:
        res = grounder.ground(name, organisms=["9606"])
        if res:
            rv[identifier] = prefix + ":" + res[0].term.id

    total = idx.sum()
    n = len(rv)
    print(f"{prefix.capitalize()} got {n:,}/{total:,} ({n / total:.1%}) groundings")
    return rv


pid_mapping = get_mappings("pid", "PID via Pathway Commons")
reactome_mapping = get_mappings("reactome", "Reactome via Pathway Commons")

[pid] mapping:   0%|          | 0.00/211 [00:00<?, ?name/s]

Pid species count: Counter({None: 211})


Grounding:   0%|          | 0.00/220 [00:00<?, ?it/s]

Pid got 193/220 (87.7%) groundings


[reactome] mapping:   0%|          | 0.00/22.0k [00:00<?, ?name/s]

Reactome species count: Counter({'9606': 2610, '10090': 1717, '9031': 1707, '10116': 1704, '9913': 1698, '7955': 1678, '9823': 1662, '9615': 1659, '8364': 1581, '7227': 1479, '6239': 1308, '44689': 988, '4896': 821, '4932': 815, '5833': 600, '1773': 13})


Grounding:   0%|          | 0.00/1.31k [00:00<?, ?it/s]

Reactome got 1,096/1,308 (83.8%) groundings


## Compare

An all-by-all comparison is done to see how many nodes are overlapping between each resource that appear in Hetionet.

In [14]:
mappings = [
    ("reactome", reactome_mapping),
    ("go", go_mapping),
    ("wikipathways", wikipathways_mapping),
    ("pid", pid_mapping),
]

merge_graph = nx.Graph()

for (p1, d1), (p2, d2) in itt.combinations(mappings, 2):
    label = f"{p1} - {p2}"
    it = tqdm(
        itt.product(d1.items(), d2.items()),
        desc=label,
        total=len(d1) * len(d2),
        unit_scale=True,
        unit="pair",
    )
    c = 0
    for (hetio_id_1, curie_1), (hetio_id_2, curie_2) in it:
        if true_graph.has_edge(curie_1, curie_2):
            merge_graph.add_edge(hetio_id_1, hetio_id_2)
            c += 1
    tqdm.write(f"{label} had {c}")

reactome - go:   0%|          | 0.00/12.5M [00:00<?, ?pair/s]

reactome - go had 95


reactome - wikipathways:   0%|          | 0.00/322k [00:00<?, ?pair/s]

reactome - wikipathways had 42


reactome - pid:   0%|          | 0.00/212k [00:00<?, ?pair/s]

...

go - wikipathways had 34


go - pid:   0%|          | 0.00/2.20M [00:00<?, ?pair/s]

go - pid had 0


wikipathways - pid:   0%|          | 0.00/56.7k [00:00<?, ?pair/s]

wikipathways - pid had 0


In [15]:
n_components = len(list(nx.connected_components(merge_graph)))
n_nodes = merge_graph.number_of_nodes()
duplicates = n_nodes - n_components

print(
    f"There are {n_nodes} that participate in duplicates, "
    f"forming {n_components} connected components in the mapping graph. "
    f"Therefore, {duplicates} nodes for pathways are duplicates."
)

There are 318 that participate in duplicates, forming 150 connected components in the mapping graph. Therefore, 168 nodes for pathways are duplicates.
